In [13]:
import numpy as np

rng = np.random.default_rng()

def two_body_decay(mB, m1, m2):
    """Energies and momentum magnitude in parent rest frame."""
    E1 = (mB**2 + m1**2 - m2**2) / (2*mB)
    E2 = (mB**2 - m1**2 + m2**2) / (2*mB)
    p  = np.sqrt(max(E1**2 - m1**2, 0.0))
    return E1, E2, p

def boost_general(E, p, beta, gamma):
    """Boost 4-vector (E, p) using beta (1D or 3D)"""
    beta = np.asarray(beta)

    if beta.ndim == 1:
        beta = beta[np.newaxis, :]  # shape (1, 3), so it broadcasts

    if p.ndim == 1:
        p = p[np.newaxis, :]  # shape (1, 3)

    beta2 = np.sum(beta**2, axis=-1)
    mask = beta2 < 1e-10  # No boost if β ≈ 0
    gamma = np.where(mask, 1.0, gamma)
    
    # Dot product and parallel component
    beta_dot_p = np.sum(p * beta, axis=-1)
    p_parallel = np.where(mask[:, None], 0.0, 
                         ((gamma - 1) / beta2)[:, None] * beta_dot_p[:, None] * beta)
    
    # Transform
    E_prime = np.where(mask, E, gamma * (E + beta_dot_p))
    p_prime = p + p_parallel + (gamma * E)[:, None] * beta
    
    return E_prime, p_prime

In [17]:
def decay_B_to_rho_a(B_line, 
                     mBd=5.279, mBs=5.367, mRho=0.775, mA=0.0, 
                     decay_rho=True, rng=np.random.default_rng()):
    """
    Replace undecayed B with B -> rho + a.
    Optionally decay rho -> pi+ pi- (rho0) or pi± pi0 (rho±).
    Treat Bs as Bd for method testing.
    
    Parameters
    ----------
    B_line : str
        One line from z-decay-products-B.txt (9 columns).
    mBd, mBs, mRho, mA : float
        Masses of Bd, Bs, rho, axion-like particle [GeV].
    decay_rho : bool
        If True, decay rho into pions; otherwise keep rho stable.
    rng : np.random.Generator
        Random number generator.
    
    Returns
    -------
    list of str
        Daughters in 9-column format (pdg px py pz E x y z t).
    """
    # --- Parse B line ---
    tk = B_line.strip().split()
    pdgB, px, py, pz, E, x, y, z, t = map(float, tk)
    pdgB = int(pdgB)
    pB = np.array([px, py, pz])
    EB = E

    # Choose correct B mass
    if abs(pdgB) == 531:   # Bs → treat as Bd
        mB = mBs
        decay_channel = "rho0"
    elif abs(pdgB) == 511: # Bd
        mB = mBd
        decay_channel = "rho0"
    elif pdgB == 521:      # B+
        mB = mBd
        decay_channel = "rho+"
    elif pdgB == -521:     # B-
        mB = mBd
        decay_channel = "rho-"
    else:
        # fallback: assume Bd
        mB = mBd
        decay_channel = "rho0"

    # --- B rest-frame decay: B -> rho + a ---
    E_rho_star, E_a_star, p_star = two_body_decay(mB, mRho, mA)
    costh, phi = 2*rng.random()-1, 2*np.pi*rng.random()
    sinth = np.sqrt(1-costh**2)
    n = np.array([sinth*np.cos(phi), sinth*np.sin(phi), costh])
    p_rho_star, p_a_star = p_star*n, -p_star*n

    betaB = pB/EB if EB > 0 else np.zeros(3)
    gammaB = EB/mB

    E_rho, p_rho = boost_general(np.array([E_rho_star]), 
                                 p_rho_star[np.newaxis,:], 
                                 betaB[np.newaxis,:], 
                                 np.array([gammaB]))
    E_a, p_a = boost_general(np.array([E_a_star]), 
                             p_a_star[np.newaxis,:], 
                             betaB[np.newaxis,:], 
                             np.array([gammaB]))
    E_rho, p_rho = E_rho[0], p_rho[0]
    E_a, p_a = E_a[0], p_a[0]

    daughters = []

    if decay_rho:
        mpi, mpi0 = 0.13957, 0.134977
        if decay_channel == "rho0":
            m1, m2, pdg1, pdg2 = mpi, mpi, 211, -211
        elif decay_channel == "rho+":
            m1, m2, pdg1, pdg2 = mpi, mpi0, 211, 111
        elif decay_channel == "rho-":
            m1, m2, pdg1, pdg2 = mpi, mpi0, -211, 111
        else:
            m1, m2, pdg1, pdg2 = mpi, mpi, 211, -211

        E1_star, E2_star, p_star = two_body_decay(mRho, m1, m2)
        costh, phi = 2*rng.random()-1, 2*np.pi*rng.random()
        sinth = np.sqrt(1-costh**2)
        n = np.array([sinth*np.cos(phi), sinth*np.sin(phi), costh])
        p1_star, p2_star = p_star*n, -p_star*n

        beta_rho = p_rho/E_rho
        gamma_rho = E_rho/mRho

        E1, p1 = boost_general(np.array([E1_star]), 
                               p1_star[np.newaxis,:], 
                               beta_rho[np.newaxis,:], 
                               np.array([gamma_rho]))
        E2, p2 = boost_general(np.array([E2_star]), 
                               p2_star[np.newaxis,:], 
                               beta_rho[np.newaxis,:], 
                               np.array([gamma_rho]))
        E1, p1 = E1[0], p1[0]
        E2, p2 = E2[0], p2[0]

        daughters.append((pdg1, (E1, *p1)))
        daughters.append((pdg2, (E2, *p2)))
    else:
        daughters.append((113, (E_rho, *p_rho)))  # rho^0 (stable)

    # --- Invisible axion-like particle ---
    daughters.append((999999, (E_a, *p_a)))

    # --- Return daughters with B’s production vertex ---
    out_lines = []
    for pdg, (E, px, py, pz) in daughters:
        out_lines.append(
            f"{pdg} {px:.6f} {py:.6f} {pz:.6f} {E:.6f} {x:.6f} {y:.6f} {z:.6f} {t:.6f}"
        )
    return out_lines



In [18]:
def inject_BSM(z_merged="z-decay-products-merged.txt",
               z_B="B-data.txt",
               out_file="z-decay-products-BSM.txt",
               decay_rho=True):
    """
    Take merged SM products + one undecayed B from B-data.txt,
    inject Bd->rho a, and output full merged events.
    All non-Bd events are discarded.

    Parameters
    ----------
    z_merged : str
        Path to SM merged file (non-B products + all other B decays).
    z_B : str
        Path to undecayed B file (one per event).
    out_file : str
        Path to write full events with Bd->rho a decay.
    decay_rho : bool
        If True, decay rho into pions, else keep rho stable.
    """

    counts = {"Bd": 0, "skipped": 0}

    with open(z_merged) as f_main, open(z_B) as f_B, open(out_file, "w") as fout:
        for ev_id, (line_main, line_B) in enumerate(zip(f_main, f_B)):
            line_main = line_main.strip()
            line_B = line_B.strip()

            # --- skip empty lines ---
            if not line_B:
                counts["skipped"] += 1
                continue

            # --- identify B type ---
            pdgB = int(line_B.split()[0])
            if abs(pdgB) != 511:  # not a Bd → skip event
                counts["skipped"] += 1
                continue

            # --- prepare daughters ---
            bsm_daughters = decay_B_to_rho_a(line_B, decay_rho=decay_rho)

            # --- collect event ---
            particles = []
            if line_main:
                particles.append(line_main)
            particles.extend(bsm_daughters)

            fout.write(" ".join(particles) + "\n")
            counts["Bd"] += 1

            if (ev_id+1) % 10000 == 0:
                print(f"[INFO] Processed {ev_id+1:,} events "
                      f"(kept {counts['Bd']:,}, skipped {counts['skipped']:,})")

    # --- summary ---
    print(f"\n[INFO] Wrote Bd-only BSM-injected events to {out_file}")
    print("=== Injection summary ===")
    print(f"   Bd kept : {counts['Bd']:,}")
    print(f"   Skipped : {counts['skipped']:,}")
    print(f"   Total   : {counts['Bd'] + counts['skipped']:,}")


In [19]:
inject_BSM()

[INFO] Processed 30,000 events (kept 14,686, skipped 15,314)
[INFO] Processed 60,000 events (kept 29,496, skipped 30,504)
[INFO] Processed 90,000 events (kept 44,046, skipped 45,954)
[INFO] Processed 110,000 events (kept 53,785, skipped 56,215)
[INFO] Processed 170,000 events (kept 83,152, skipped 86,848)
[INFO] Processed 200,000 events (kept 97,915, skipped 102,085)
[INFO] Processed 240,000 events (kept 117,612, skipped 122,388)
[INFO] Processed 290,000 events (kept 142,314, skipped 147,686)
[INFO] Processed 300,000 events (kept 147,240, skipped 152,760)
[INFO] Processed 310,000 events (kept 152,121, skipped 157,879)
[INFO] Processed 340,000 events (kept 166,779, skipped 173,221)
[INFO] Processed 350,000 events (kept 171,681, skipped 178,319)
[INFO] Processed 360,000 events (kept 176,588, skipped 183,412)
[INFO] Processed 370,000 events (kept 181,450, skipped 188,550)
[INFO] Processed 390,000 events (kept 191,252, skipped 198,748)
[INFO] Processed 400,000 events (kept 196,159, skipped